In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggledays-flying-circus/sample_submission.csv
/kaggle/input/kaggledays-flying-circus/hobbies.csv
/kaggle/input/kaggledays-flying-circus/train.csv
/kaggle/input/kaggledays-flying-circus/test.csv
/kaggle/input/kaggledays-flying-circus/images/images/e461f5a7c3d9.jpg
/kaggle/input/kaggledays-flying-circus/images/images/fbddd5f842b6.jpg
/kaggle/input/kaggledays-flying-circus/images/images/b4b35bca690a.jpg
/kaggle/input/kaggledays-flying-circus/images/images/7c1cab944e1c.jpg
/kaggle/input/kaggledays-flying-circus/images/images/a0c970332244.jpg
/kaggle/input/kaggledays-flying-circus/images/images/b8ec92db3761.jpg
/kaggle/input/kaggledays-flying-circus/images/images/25dd69134145.jpg
/kaggle/input/kaggledays-flying-circus/images/images/1a59859b86ea.jpg
/kaggle/input/kaggledays-flying-circus/images/images/f78ca7681a80.jpg
/kaggle/input/kaggledays-flying-circus/images/images/a29edb78d08b.jpg
/kaggle/input/kaggledays-flying-circus/images/images/62ae1c64e2ea.jpg
/kaggle/input/kaggled

In [55]:
os.listdir("../input/")

['kaggledays-flying-circus']

In [56]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

In [117]:
class Config:
    image_path = '../input/kaggledays-flying-circus/images/images'
    batch_size = 128
    img_height = 224
    img_width = 224
    val_size = int(len(pd.read_csv('../input/kaggledays-flying-circus/train.csv')) * 0.1)
    epoch = 1
    ft_epoch = 1
    base_learning_rate = 0.0001

In [118]:
train_df = pd.read_csv('../input/kaggledays-flying-circus/train.csv')
test_df = pd.read_csv('../input/kaggledays-flying-circus/test.csv')
hobbies_df = pd.read_csv('../input/kaggledays-flying-circus/hobbies.csv')
sample_submission = pd.read_csv('../input/kaggledays-flying-circus/sample_submission.csv')

In [119]:
train_df.head()

,id,img_path,celebrity,age
0,1f27724cc1e1,1f27724cc1e1.jpg,Actress,27
1,f987835a9d20,f987835a9d20.jpg,Actress,25
2,cbc36c5c8368,cbc36c5c8368.jpg,Actress,21
3,f9db1e5514f5,f9db1e5514f5.jpg,Actress,26
4,c8611e89f4e6,c8611e89f4e6.jpg,Actor,38


In [120]:
hobbies_df["st"] = [s.split("-")[0] for s in hobbies_df["age"]]
hobbies_df["end"] = [s.split("-")[1] for s in hobbies_df["age"]]
hobbies_df["st"] = hobbies_df["st"].astype("int")
hobbies_df["end"] = hobbies_df["end"].astype("int")
allres = []
for num,row in hobbies_df.iterrows():
    for age in range(row["st"],row["end"]+1):
        allres.append([age,row["celebrity"],row["hobbies"]])
allresdf = pd.DataFrame(allres)
allresdf.columns = ["age", "celebrity", "hobbies"]
train_df = pd.merge(train_df, allresdf, on=["age","celebrity"], how="left")

In [121]:
train_df['celebrity'] = train_df['celebrity'].astype('category').cat.codes

In [122]:
train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df))
train_ds = train_ds.shuffle(buffer_size=10)
train_ds = train_ds.skip(Config.val_size*6)
val_ds = train_ds.take(Config.val_size)
test_ds = tf.data.Dataset.from_tensor_slices(dict(test_df))

In [123]:
for batch in train_ds:
    print(batch)
    break

{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'd292c93f6f85'>, 'img_path': <tf.Tensor: shape=(), dtype=string, numpy=b'd292c93f6f85.jpg'>, 'celebrity': <tf.Tensor: shape=(), dtype=int8, numpy=0>, 'age': <tf.Tensor: shape=(), dtype=int64, numpy=52>, 'hobbies': <tf.Tensor: shape=(), dtype=string, numpy=b'television'>}


In [124]:
def get_data(item):
    img = tf.io.read_file(Config.image_path+'/'+item['img_path'])
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (Config.img_height, Config.img_width))
    
    return img, (item['celebrity'], item['age'])

def get_data_test(item):
    img = tf.io.read_file(Config.image_path+'/'+item['img_path'])
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (Config.img_height, Config.img_width))
    return img, item['img_path']

In [125]:
train_ds = train_ds.map(get_data)
val_ds = val_ds.map(get_data)
test_ds = test_ds.map(get_data_test)

In [126]:
train_ds = train_ds.cache()
val_ds = val_ds.cache()

In [127]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.batch(Config.batch_size)
train_ds = train_ds.prefetch(AUTOTUNE)
train_ds = train_ds.repeat(Config.batch_size)

val_ds = val_ds.batch(Config.batch_size)
val_ds = val_ds.prefetch(AUTOTUNE)
val_ds = val_ds.repeat(Config.batch_size)

test_ds = test_ds.batch(Config.batch_size)
test_ds = test_ds.prefetch(AUTOTUNE)

In [128]:
# def prepareImages(data, m, dataset):
#     print("Preparing images")
#     X_train = np.zeros((m, 100, 100, 3))
#     count = 0
    
#     for fig in data['img_path']:
#         #load images into images of size 100x100x3
#         img = image.load_img("/kaggle/input/kaggledays-flying-circus/images/images/{0}".format(fig), target_size=(100, 100, 3))
#         x = image.img_to_array(img)
#         x = preprocess_input(x)

#         X_train[count] = x
#         if (count%500 == 0):
#             print("Processing image: ", count+1, ", ", fig)
#         count += 1
    
#     return X_train

In [129]:
# X = prepareImages(train_df, train_df.shape[0], "train")
# X /= 255

In [130]:
# y = train_df[['celebrity','age']]

In [131]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [132]:
visible = Input((224, 224,3))
hidden_conv_1 = Conv2D(32,(3,3), activation='relu', padding='same')(visible)
hidden_maxpool_1 = MaxPool2D()(hidden_conv_1)
hidden_b_norm_1 = BatchNormalization()(hidden_maxpool_1)

hidden_conv_2 = Conv2D(64,(3,3), activation='relu', padding='same')(hidden_b_norm_1)
hidden_maxpool_2 = MaxPool2D()(hidden_conv_2)
hidden_b_norm_2 = BatchNormalization()(hidden_maxpool_2)

hidden_gAvg_1 = GlobalAveragePooling2D()(hidden_b_norm_2)
hidden_dense_1 = Dense(64, activation='relu')(hidden_gAvg_1)
out_class = Dense(1, name='cls', activation='sigmoid')(hidden_dense_1)
out_reg = tf.keras.layers.Dense(1, name='reg')(hidden_dense_1)




In [133]:
model = tf.keras.Model(inputs=visible, outputs=[out_class, out_reg])

In [134]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=Config.base_learning_rate),
              loss={'cls': tf.keras.losses.BinaryCrossentropy(from_logits=False), 'reg': tf.keras.losses.MeanSquaredError()},
              metrics={'cls': 'accuracy', 'reg': 'mean_absolute_error'})

In [135]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 224, 224, 32) 896         input_8[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_14 (MaxPooling2D) (None, 112, 112, 32) 0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 112, 112, 32) 128         max_pooling2d_14[0][0]           
____________________________________________________________________________________________

In [ ]:
history = model.fit(train_ds,
                    epochs=Config.epoch,
                    validation_data=val_ds)

   76/10112 [..............................] - ETA: 9:15:46 - loss: 1490.6378 - cls_loss: 0.7085 - reg_loss: 1489.9293 - cls_accuracy: 0.5105 - reg_mean_absolute_error: 36.4395

In [ ]:
acc = history.history['cls_accuracy']
val_acc = history.history['val_cls_accuracy']

loss = history.history['cls_loss']
val_loss = history.history['val_cls_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_preds = model.predict(test_ds)
test_df_sumb = test_df.copy()
test_df_sumb['celebrity'] = tf.where(test_preds[0] > 0.5, 1, 0)
test_df_sumb['celebrity'] = test_df_sumb['celebrity'].replace({1: 'Actress', 0: 'Actor'})
test_df_sumb['age'] = test_preds[1]
test_df_sumb['age'] = test_df_sumb['age'].astype(int)
def map_fn_hobbies(x):
    try:
        result = hobbies_df[(hobbies_df.st <= x['age']) & (hobbies_df.end >= x['age']) & (hobbies_df['celebrity'] == x['celebrity'])]['hobbies'].item()
    except:
        result = 'fashion'
    return result
test_df_sumb['hobbies'] =  test_df_sumb.apply(map_fn_hobbies,  axis=1)
test_df_sumb.head(20)

In [ ]:
sample_submission['hobbies'] = test_df_sumb['hobbies']
sample_submission.to_csv('sample_sumb.csv', index=False)